In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode

In [2]:
#Veri çekme aşaması; çekilen verileri url,fiyat,özellik başlığı ve verisi şeklinde listelere bölme
url_list = []
prices_list = []
propTitles = []
propValues = []

In [3]:
#Sayfa numaraları 1'den 25'e kadar dönerek Trendyol bisiklet verilerini çekme
for i in range(1, 26):
    url = "https://www.trendyol.com/dag-bisikleti-y-s7377?pi=" + str(i) 
    r = requests.get(url)
    source = BeautifulSoup(r.content, "lxml")

    urls = source.find_all("div", attrs={"class": "card-border"})
    for url in urls:
        url_bicycle = "https://www.trendyol.com/" + url.a.get("href")
        url_list.append(url_bicycle)
        #print(url_bicycle)

        r_bicycle = requests.get(url_bicycle)
        source_bicycle = BeautifulSoup(r_bicycle.content, "lxml")

        properties = source_bicycle.find_all("li", attrs={"class": "detail-attr-item"})
        for prop in properties:
            prop_title = prop.find("span").text
            prop_value = prop.find("b").text
            propTitles.append(prop_title)
            propValues.append(prop_value)

    prices = source.find_all("div", attrs={"class": "prc-box-dscntd"})
    for price in prices:
        prices_list.append(price.text)
        
print(str(len(url_list)) + " Adet link bulundu.")
print(str(len(prices_list)) + " Adet fiyat bulundu.")
print(str(len(propTitles)) + " Adet özellik başlığı bulundu.")
print(str(len(propValues)) + " Adet özellik verisi bulundu.")


600 Adet link bulundu.
600 Adet fiyat bulundu.
4022 Adet özellik başlığı bulundu.
4022 Adet özellik verisi bulundu.


In [4]:
df = pd.DataFrame(columns=np.unique(np.array(propTitles)))
df["url"] = url_list
df["price"] = prices_list

In [20]:
def clean_text(text):
    # Text üzerinde düzenleme işlemleri yapılabilir
    cleaned_text = text.replace('Ä', 'A').replace('Ã', 'C').replace('¶', 'o').replace('Ş', 'S').replace('Ä±', 'ı').replace('Ä', 'g').replace('Å', 'U').replace('¼', 'u').replace('„', 'i').replace('©', 'e').replace('Äğ','ğ')
    return cleaned_text
df.columns = [clean_text(col) for col in df.columns]
df.head()


,Unnamed: 0,Bisiklet Tipi,Fren Tipi,Garanti SCuresi,Garanti Süresi,Jant,Kadro Boyu,Materyal,Renk,Tekerlek SayA±sA±,Tekerlek Sayısı,Vites,Vites Sistemi,url,price
0,0,Dağ Bisikleti,V Fren,NaN,NaN,24.0,44 cm,Çelik,Beyaz,NaN,NaN,21 vites,NaN,https://www.trendyol.com//belderia/tec-strong-...,2.649 TL
1,1,Dağ Bisikleti,Hidrolik Fren,NaN,2 Yıl,27.5,48 cm,NaN,Mavi,NaN,NaN,21 vites,NaN,https://www.trendyol.com//kron/xc100-27-5-jant...,8.200 TL
2,2,Dağ Bisikleti,Mekanik Disk Fren,NaN,2 Yıl,26.0,NaN,Çelik,Çok Renkli,NaN,2.0,21 vites,Shimano LX,https://www.trendyol.com//daafu/sxc-150-26-jan...,5.999 TL
3,3,Dağ Bisikleti,Hidrolik Disk Fren,NaN,2 Yıl,26.0,46 cm,Alüminyum,Yeşil,NaN,2.0,21 vites,NaN,https://www.trendyol.com//corelli/zengo-fat-bi...,16.999 TL
4,4,Dağ Bisikleti,Hidrolik Disk Fren,NaN,2 Yıl,26.0,46 cm,Çelik,Siyah,NaN,2.0,21 vites,Shimano XT,https://www.trendyol.com//salcano/lion-26-md-d...,"7.249,90 TL"


In [6]:
for i in range(len(url_list)):
    url = df['url'].loc[i]
    r = requests.get(url)
    source = BeautifulSoup(r.content, "lxml", from_encoding="utf-8")

    properties = source.find_all("li", attrs={"class": "detail-attr-item"})
    for prop in properties:
        prop_title = clean_text(prop.find("span").text)
        prop_value = prop.find("b").text

        # DataFrame'e değeri eklerken düzeltilmiş başlığı kullanma
        try:
            df[prop_title].loc[i] = prop_value
        except KeyError:
            print(f"Sorunlu başlık: {prop_title}")

In [13]:
df.head()

,Bisiklet Tipi,Fren Tipi,Garanti SCuresi,Garanti Süresi,Jant,Kadro Boyu,Materyal,Renk,Tekerlek SayA±sA±,Tekerlek Sayısı,Vites,Vites Sistemi,url,price
0,Dağ Bisikleti,V Fren,NaN,NaN,24,44 cm,Çelik,Beyaz,NaN,NaN,21 vites,NaN,https://www.trendyol.com//belderia/tec-strong-...,2.649 TL
1,Dağ Bisikleti,Hidrolik Fren,NaN,2 Yıl,27.5,48 cm,NaN,Mavi,NaN,NaN,21 vites,NaN,https://www.trendyol.com//kron/xc100-27-5-jant...,8.200 TL
2,Dağ Bisikleti,Mekanik Disk Fren,NaN,2 Yıl,26,NaN,Çelik,Çok Renkli,NaN,2,21 vites,Shimano LX,https://www.trendyol.com//daafu/sxc-150-26-jan...,5.999 TL
3,Dağ Bisikleti,Hidrolik Disk Fren,NaN,2 Yıl,26,46 cm,Alüminyum,Yeşil,NaN,2,21 vites,NaN,https://www.trendyol.com//corelli/zengo-fat-bi...,16.999 TL
4,Dağ Bisikleti,Hidrolik Disk Fren,NaN,2 Yıl,26,46 cm,Çelik,Siyah,NaN,2,21 vites,Shimano XT,https://www.trendyol.com//salcano/lion-26-md-d...,"7.249,90 TL"


In [14]:
# DataFrame'i Excel dosyasına kaydetme
df.to_excel("output.xlsx")


In [15]:
pwd

'C:\\Users\\LENOVO'

In [10]:
current_directory = 'C:\\Users\\LENOVO'

# Dosyanın adını ve uzantısını ekleyin
file_name = 'output.xlsx'

# Dosyanın tam yolunu oluşturun
file_path = f'{current_directory}\\{file_name}'

print(file_path)


C:\Users\LENOVO\output.xlsx


In [16]:
df = pd.read_excel(file_path)

print(df.head())

   Unnamed: 0  Bisiklet Tipi           Fren Tipi  Garanti SCuresi  \
0           0  Dağ Bisikleti              V Fren              NaN   
1           1  Dağ Bisikleti       Hidrolik Fren              NaN   
2           2  Dağ Bisikleti   Mekanik Disk Fren              NaN   
3           3  Dağ Bisikleti  Hidrolik Disk Fren              NaN   
4           4  Dağ Bisikleti  Hidrolik Disk Fren              NaN   

  Garanti Süresi  Jant Kadro Boyu   Materyal        Renk  Tekerlek SayA±sA±  \
0            NaN  24.0      44 cm      Çelik       Beyaz                NaN   
1          2 Yıl  27.5      48 cm        NaN        Mavi                NaN   
2          2 Yıl  26.0        NaN      Çelik  Çok Renkli                NaN   
3          2 Yıl  26.0      46 cm  Alüminyum       Yeşil                NaN   
4          2 Yıl  26.0      46 cm      Çelik       Siyah                NaN   

   Tekerlek Sayısı     Vites Vites Sistemi  \
0              NaN  21 vites           NaN   
1              NaN

In [21]:
df.to_csv("bicycle_data.csv",index = False)